## Loading libraries

In [76]:
import json.decoder

import numpy as np
import pandas as pd
import sklearn as sk
import json as json
# import seaborn as sns
import matplotlib
import pickle as pk
import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer

print('numpy version:', np.__version__)
print('pandas version:', pd.__version__)
print('scikit-learn version:', sk.__version__)
print('json version:', json.__version__)
print('matplotlib version:', matplotlib.__version__)

numpy version: 1.23.3
pandas version: 1.5.0
scikit-learn version: 1.1.2
json version: 2.0.9
matplotlib version: 3.6.0


# Loading data

In [77]:
#DataSets
# df = pd.read_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv')
RAW_recipes = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\RAW_recipes.csv")
RAW_interactions = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\RAW_interactions.csv")
pkl_recipes = pd.read_pickle(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\ingr_map.pkl")

pp_recipes = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\PP_recipes.csv")
pp_users = pd.read_csv(r"C:\Users\evald\Desktop\Ai minor\dataset\Recipes\archive\PP_users.csv")



In [78]:
RAW_recipes

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


#### Nutritional values are aranges as
[calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV) , and carbohydrates (PDV)]

In [79]:

RAW_recipes_CollumnSetUp= ['name', 'id', 'minutes','nutrition','contributor_id', 'submitted','n_steps', 'steps', 'description', 'ingredients', 'n_ingredients']
RAW_recipes = RAW_recipes[RAW_recipes_CollumnSetUp]

In [80]:
RAW_recipes

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


# Cleaning data

### Here we get important components from different datasets

In [81]:

recipesKG = pp_recipes.merge(right=RAW_recipes, left_on="id", right_on="id")

# recipesKG = recipesKG[["id", "name", "submitted", "ingredient_ids", "ingredients", "n_ingredients"]]
recipesKG = recipesKG[["id", "ingredient_ids"]]
recipesKG = recipesKG.rename({"id":"recipe_id"}, axis="columns")
recipesKG.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178265 entries, 0 to 178264
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   recipe_id       178265 non-null  int64 
 1   ingredient_ids  178265 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.1+ MB


In [82]:
# Get ingredients id with uniques ingredient name, no duplicates
ingredient_df = pkl_recipes.copy(deep=True)
ingredient_df = ingredient_df.rename({"id":"ingredient_id","replaced":"ingredient_name"}, axis="columns")
ingredient_df = ingredient_df[["ingredient_id", "ingredient_name"]]
ingredient_df = ingredient_df.drop_duplicates(ignore_index=True)
ingredient_df.head(10)

,ingredient_id,ingredient_name
0,4308,lettuce
1,2744,french vanilla pudding and pie filling mix
2,6843,stove top stuffing mix
3,1910,cream cheese
4,1168,cheddar
5,5811,radicchio
6,5205,pasta sauce
7,7233,tomato sauce
8,6482,shredded three cheese
9,4836,mozzarella


Getting user review ratings on recipes set

In [83]:
RAW_interactions=RAW_interactions.drop(['user_id','date','review'],axis=1)
RAW_interactions.head()

,recipe_id,rating
0,40893,4
1,40893,5
2,44394,4
3,85009,5
4,85009,5


## Unique ingredients within the recipe set

In [84]:
uniqueIngredients = pkl_recipes.groupby(by=['replaced'],as_index=False )['id'].count()
uniqueIngredientsDF = pd.DataFrame(uniqueIngredients)
uniqueIngredientsDF.rename(columns={'replaced':'ingredient_name','id':'frequency_count'},inplace=True)

In [85]:
uniqueIngredientsDF.head(10)

,ingredient_name,frequency_count
0,'s baking chocolate,1
1,'s chocolate chip,1
2,'s hugs chocolate,1
3,'s sauce,1
4,'s sour cream,1
5,'s spicy seasoning,1
6,'s syrup,1
7,'s teriyaki sauce,1
8,10 bean soup mix,1
9,10-minute success rice,1


In [86]:
uniqueIngredientsDF.sort_values(by=['frequency_count'], ascending=False)

,ingredient_name,frequency_count
2200,dressing,163
6324,sausage,154
1316,chile,99
912,cake mix,85
1168,cheddar,84
...,...,...
3011,frozen english pea,1
3010,frozen edamame,1
3009,frozen dough roll,1
3008,frozen dinner roll,1


# Merging the tables to combine the important features

In [87]:
#Store average rating of meals
AvgInteractions = RAW_interactions['rating'].mean()

#Group seperate meal ratings together and get average meal score equally prevent recipe_id from being an index
newRatingSet=RAW_interactions.groupby(['recipe_id'])['rating'].agg(['mean','count'])
print(newRatingSet.head(10))
newRatingSet.reset_index(inplace=True)


#Rename columns
newRatingSet.rename(columns={'mean':'average_rating','count':'votes'},inplace=True)
newRatingSet.head(10)

               mean  count
recipe_id                 
38         4.250000      4
39         3.000000      1
40         4.333333      9
41         4.500000      2
43         1.000000      1
45         2.666667      3
46         5.000000      2
48         1.000000      2
49         4.350000     20
50         4.500000      2


,recipe_id,average_rating,votes
0,38,4.250000,4
1,39,3.000000,1
2,40,4.333333,9
3,41,4.500000,2
4,43,1.000000,1
5,45,2.666667,3
6,46,5.000000,2
7,48,1.000000,2
8,49,4.350000,20
9,50,4.500000,2


In [88]:
#Merge ratings dataframe with recipe Dataframe
food_df = pd.merge(RAW_recipes, newRatingSet, left_on='id',right_on='recipe_id')

In [89]:
food_df.shape

(231637, 14)

In [90]:
food_df

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients,recipe_id,average_rating,votes
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,137739,5.0,3
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,31490,3.5,4
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,112140,4.0,1
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,59389,4.5,2
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,44061,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,486161,5.0,6
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,493372,5.0,1
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,308080,5.0,5
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,298512,1.0,1


In [91]:
#merge the recipe ingredient id array
#
food_df = pd.merge(food_df, recipesKG[['recipe_id', 'ingredient_ids']], on='recipe_id', how='left')

In [92]:
food_df

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients,recipe_id,average_rating,votes,ingredient_ids
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,137739,5.0,3,"[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,31490,3.5,4,"[5481, 6324, 2499, 4717, 6276, 1170]"
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,112140,4.0,1,NaN
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,59389,4.5,2,"[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,44061,5.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,486161,5.0,6,NaN
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,493372,5.0,1,NaN
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,308080,5.0,5,"[3627, 4623, 2148, 905, 7049, 6270, 590, 2822]"
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,298512,1.0,1,NaN


In [93]:
#Show the number of votes possesed by the top 10% meals, based on number of votes
q=newRatingSet['votes'].quantile(0.9)
print(q)

9.0


In [94]:
food_df

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients,recipe_id,average_rating,votes,ingredient_ids
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,137739,5.0,3,"[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,31490,3.5,4,"[5481, 6324, 2499, 4717, 6276, 1170]"
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,112140,4.0,1,NaN
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,59389,4.5,2,"[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,44061,5.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,486161,5.0,6,NaN
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,493372,5.0,1,NaN
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,308080,5.0,5,"[3627, 4623, 2148, 905, 7049, 6270, 590, 2822]"
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,298512,1.0,1,NaN


# Quality of the Data Sets

In [95]:
MissingValues = food_df.isnull().sum()
MissingValues

name                  1
id                    0
minutes               0
nutrition             0
contributor_id        0
submitted             0
n_steps               0
steps                 0
description        4979
ingredients           0
n_ingredients         0
recipe_id             0
average_rating        0
votes                 0
ingredient_ids    53372
dtype: int64

These methods will clean up missing values withing describtions and names by replacing them with a string.

In [96]:
def cleanNullValName():
    food_df['name'].fillna("unknown",inplace=True)
    response = food_df.isnull().sum()
    return response

def cleanNullValDescribtion():
    if(food_df['description'].isnull().sum() > 1):
        food_df['description'].fillna("unknown",inplace=True)
        response = food_df.isnull().sum()
        response
        print("null values removed")
    else:
        print("no null values")

def cleanNullValIngredients():
    if(food_df['ingredient_ids'].isnull().sum() > 1):
        food_df['ingredient_ids'].fillna("unknown",inplace=True)
        response = food_df.isnull().sum()
        response
        print("null values removed")
    else:
        print("no null values")

Clean describtions

In [97]:
cleanNullValName()
cleanNullValDescribtion()
cleanNullValIngredients()

null values removed
null values removed


## Now that the missing values are cleaned we can proceed with editing preferences

In [98]:
food_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            231637 non-null  object 
 1   id              231637 non-null  int64  
 2   minutes         231637 non-null  int64  
 3   nutrition       231637 non-null  object 
 4   contributor_id  231637 non-null  int64  
 5   submitted       231637 non-null  object 
 6   n_steps         231637 non-null  int64  
 7   steps           231637 non-null  object 
 8   description     231637 non-null  object 
 9   ingredients     231637 non-null  object 
 10  n_ingredients   231637 non-null  int64  
 11  recipe_id       231637 non-null  int64  
 12  average_rating  231637 non-null  float64
 13  votes           231637 non-null  int64  
 14  ingredient_ids  231637 non-null  object 
dtypes: float64(1), int64(7), object(7)
memory usage: 28.3+ MB


### To save additional memory I can convert the int data types from 64 to 32

In [99]:
def convertDataSetsToInt32():
    food_df.id = food_df.id.astype('int32')
    food_df.contributor_id = food_df.contributor_id.astype('int32')
    food_df.minutes = food_df.minutes.astype('int32')
    food_df.n_steps = food_df.n_steps.astype('int32')
    food_df.n_ingredients = food_df.n_ingredients.astype('int32')
    food_df.recipe_id = food_df.recipe_id.astype('int32')
    food_df.votes = food_df.votes.astype('int32')
    food_df.info()

convertDataSetsToInt32()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   name            231637 non-null  object 
 1   id              231637 non-null  int32  
 2   minutes         231637 non-null  int32  
 3   nutrition       231637 non-null  object 
 4   contributor_id  231637 non-null  int32  
 5   submitted       231637 non-null  object 
 6   n_steps         231637 non-null  int32  
 7   steps           231637 non-null  object 
 8   description     231637 non-null  object 
 9   ingredients     231637 non-null  object 
 10  n_ingredients   231637 non-null  int32  
 11  recipe_id       231637 non-null  int32  
 12  average_rating  231637 non-null  float64
 13  votes           231637 non-null  int32  
 14  ingredient_ids  231637 non-null  object 
dtypes: float64(1), int32(7), object(7)
memory usage: 22.1+ MB


In [100]:
food_df

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients,recipe_id,average_rating,votes,ingredient_ids
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,137739,5.0,3,"[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,31490,3.5,4,"[5481, 6324, 2499, 4717, 6276, 1170]"
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,112140,4.0,1,unknown
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,59389,4.5,2,"[1170, 4918, 6426, 5185, 7099, 5006, 6009, 627..."
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,44061,5.0,1,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,486161,5.0,6,unknown
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,493372,5.0,1,unknown
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,308080,5.0,5,"[3627, 4623, 2148, 905, 7049, 6270, 590, 2822]"
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,298512,1.0,1,unknown


No more missing values

In [101]:
MissingValues = food_df.isnull().sum()
MissingValues

name              0
id                0
minutes           0
nutrition         0
contributor_id    0
submitted         0
n_steps           0
steps             0
description       0
ingredients       0
n_ingredients     0
recipe_id         0
average_rating    0
votes             0
ingredient_ids    0
dtype: int64

## Removing unnecesary symbols in datasets

In [102]:
def dataSetCleanUp():
    food_df['name'] = food_df['name'].str.replace('\'s', '')
    food_df['ingredients'] = food_df['ingredients'].str.replace('(\[|\])', '')
    food_df['ingredients'] = food_df['ingredients'].str.replace('(\')', '')
    food_df['steps'] = food_df['steps'].str.replace('(\[|\])', '')
    food_df['ingredient_ids'] = food_df['ingredient_ids'].str.replace('(\[|\])', '')
    food_df['steps'] = food_df['steps'].str.replace('(\')', '')

In [103]:
dataSetCleanUp()

C:\Users\evald\AppData\Local\Temp\ipykernel_15956\3577813700.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  food_df['ingredients'] = food_df['ingredients'].str.replace('(\[|\])', '')
C:\Users\evald\AppData\Local\Temp\ipykernel_15956\3577813700.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  food_df['ingredients'] = food_df['ingredients'].str.replace('(\')', '')
C:\Users\evald\AppData\Local\Temp\ipykernel_15956\3577813700.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  food_df['steps'] = food_df['steps'].str.replace('(\[|\])', '')
C:\Users\evald\AppData\Local\Temp\ipykernel_15956\3577813700.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  food_df['ingredient_ids'] = food_df['ingredient_ids'].str.replace('(\[|\])', '')
C:\Users\evald\AppData\Local\Temp\ipykernel_15956\3577813

In [104]:
food_df.describe()

,id,minutes,contributor_id,n_steps,n_ingredients,recipe_id,average_rating,votes
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000,231637.000000,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153,222014.708984,4.346246,4.888541
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796,141206.635626,0.990806,17.532481
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000,38.000000,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000,99944.000000,4.000000,1.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000,207249.000000,4.714286,2.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000,333816.000000,5.000000,4.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000,537716.000000,5.000000,1613.000000


# Defining liked and disliked ingredients

In [105]:
IngredientPreference = food_df.__deepcopy__()

# let's define some polarising ingredients
liked_ingredients = ["cheese", "pizza", "avocado", "chicken"]
hated_ingredients = ["spinach", "mushroom", "olive", "beef", "truffle", "blue cheese"]


# let's add a "like" property
IngredientPreference["like"] = 0

# and populate our data
IngredientPreference.loc[IngredientPreference.ingredients.str.lower().str.contains("|".join(hated_ingredients)), "like"] = -1
IngredientPreference.loc[IngredientPreference.ingredients.str.lower().str.contains("|".join(liked_ingredients)), "like"] = +1

In [106]:
IngredientPreference

,name,id,minutes,nutrition,contributor_id,submitted,n_steps,steps,description,ingredients,n_ingredients,recipe_id,average_rating,votes,ingredient_ids,like
0,arriba baked winter squash mexican style,137739,55,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",47892,2005-09-16,11,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,137739,5.0,3,"7933, 4694, 4795, 3723, 840, 5006, 6270",-1
1,a bit different breakfast pizza,31490,30,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",26278,2002-06-17,9,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,31490,3.5,4,"5481, 6324, 2499, 4717, 6276, 1170",1
2,all in the kitchen chili,112140,130,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",196586,2005-02-25,6,"brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,112140,4.0,1,unknown,1
3,alouette potatoes,59389,45,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",68585,2003-04-14,11,place potatoes in a large pot of lightly salte...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,59389,4.5,2,"1170, 4918, 6426, 5185, 7099, 5006, 6009, 6270...",1
4,amish tomato ketchup for canning,44061,190,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",41706,2002-10-25,5,"mix all ingredients& boil for 2 1 / 2 hours , ...",my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,44061,5.0,1,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",227978,2012-08-29,7,"heat oil in a 4-quart dutch oven, add celery ,...",this is a delicious soup that i originally fou...,"celery, onion, green sweet pepper, garlic clov...",22,486161,5.0,6,unknown,1
231633,zydeco spice mix,493372,5,"[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1500678,2013-01-09,1,mix all ingredients together thoroughly,this spice mix will make your taste buds dance!,"paprika, salt, garlic powder, onion powder, dr...",13,493372,5.0,1,unknown,0
231634,zydeco ya ya deviled eggs,308080,40,"[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",37779,2008-06-07,7,"in a bowl , combine the mashed yolks and mayon...","deviled eggs, cajun-style","hard-cooked eggs, mayonnaise, dijon mustard, s...",8,308080,5.0,5,"3627, 4623, 2148, 905, 7049, 6270, 590, 2822",0
231635,cookies by design cookies on a stick,298512,29,"[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",506822,2008-04-15,9,place melted butter in a large mixing bowl and...,"i've heard of the 'cookies by design' company,...","butter, eagle brand condensed milk, light brow...",10,298512,1.0,1,unknown,0


# Preparing the data for plotting

# Preparing data for machine learning

# Output results
Here I will export the wanted datasets for later use

In [126]:
def compressionToOutputCsv(dataset):
    compresionThingie = dict(method='zip',
                             archive_name='results.csv')
    savedFile = dataset
    savedFile.to_csv('datasetPrep.zip',index=False,compression=compresionThingie)

In [127]:
# compressionToOutputCsv(IngredientPreference)